In [1]:
import re

import numpy as np
import pandas as pd

import folium
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

seed = 7
np.random.seed(seed=seed)

In [2]:
df = pd.read_csv('../../../input/2016 School Explorer.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1272 entries, 0 to 1271
Columns: 161 entries, Adjusted Grade to Grade 8 Math 4s - Economically Disadvantaged
dtypes: float64(5), int64(123), object(33)
memory usage: 1.6+ MB


In [4]:
df.describe()

,SED Code,District,Latitude,Longitude,Zip,Economic Need Index,Average ELA Proficiency,Average Math Proficiency,Grade 3 ELA - All Students Tested,Grade 3 ELA 4s - All Students,...,Grade 8 Math - All Students Tested,Grade 8 Math 4s - All Students,Grade 8 Math 4s - American Indian or Alaska Native,Grade 8 Math 4s - Black or African American,Grade 8 Math 4s - Hispanic or Latino,Grade 8 Math 4s - Asian or Pacific Islander,Grade 8 Math 4s - White,Grade 8 Math 4s - Multiracial,Grade 8 Math 4s - Limited English Proficient,Grade 8 Math 4s - Economically Disadvantaged
count,1.272000e+03,1272.00000,1272.000000,1272.000000,1272.000000,1247.000000,1217.000000,1217.000000,1272.000000,1272.000000,...,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000
mean,3.286698e+11,16.13522,40.734537,-73.918347,10815.720912,0.672281,2.534215,2.668956,60.569182,4.952830,...,43.841195,4.911950,0.003145,0.610063,0.947327,1.984277,0.970912,0.002358,0.159591,2.992138
std,1.222022e+10,9.24527,0.086602,0.080576,529.588875,0.210959,0.363589,0.470470,57.872496,8.300568,...,82.878780,20.792371,0.068635,3.966083,4.056007,12.841333,6.880223,0.084116,1.321195,12.694124
min,3.075000e+11,1.00000,40.507803,-74.244025,10001.000000,0.049000,1.810000,1.830000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.208752e+11,9.00000,40.669499,-73.957057,10452.000000,0.550000,2.250000,2.300000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.315004e+11,15.00000,40.722995,-73.920571,11203.000000,0.731000,2.450000,2.580000,54.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.424000e+11,24.00000,40.815632,-73.879264,11232.000000,0.841000,2.760000,2.980000,94.000000,7.000000,...,59.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.531009e+11,32.00000,40.903455,-73.708920,11694.000000,0.957000,3.930000,4.200000,356.000000,55.000000,...,652.000000,312.000000,2.000000,107.000000,71.000000,246.000000,126.000000,3.000000,33.000000,196.000000


In [193]:
df_registration = pd.read_csv('../../../input/2017-2018 SHSAT Admissions Test Offers By Sending School.csv')

In [194]:
df_merged = df.merge(df_registration[['School DBN', 
                                      'Borough', 
                                      'School Category', 
                                      'Number of students who took test']], 
                     left_on='Location Code', 
                     right_on='School DBN', 
                     how='inner')

In [195]:
df_merged['IsCommunitySchool'] = (df_merged['Community School?'] == 'Yes').astype('float')

percent_pattern = r'Percent\s.*'
percent_regex = re.compile(percent_pattern, re.IGNORECASE)

for col in df_merged.columns.values:
    if percent_regex.search(col):
        print col
        df_merged[col] = df_merged[col].astype(np.object).str.replace('%', '').astype(float)
df_merged['School Income Estimate'] = df_merged['School Income Estimate'].astype(np.object).str.replace('$', '').str.replace(',', '').str.replace('.', '').astype(float)
df_merged['Student Attendance Rate'] = df_merged['Student Attendance Rate'].astype(np.object).str.replace('%', '').astype(float)
df_merged['Percent of Students Chronically Absent'] = df_merged['Percent of Students Chronically Absent'].astype(np.object).str.replace('%', '').astype(float)

df_merged['Grade High'] = df_merged['Grade High'].astype(np.object_)
df_merged['Grade High'] = df_merged['Grade High'].str.replace('0K', '00')
df_merged['Grade High'] = df_merged['Grade High'].str.replace('PK', '00')

SPI_columns = ['Rigorous Instruction %', 'Collaborative Teachers %', 'Supportive Environment %', 
               'Effective School Leadership %', 'Strong Family-Community Ties %', 'Trust %']
for col in SPI_columns:
    df_merged[col] = df_merged[col].astype(np.object).str.replace('%', '').astype(float)

df_merged['SPI'] = df_merged['Supportive Environment %'] \
                 + 0.8 * df_merged['Rigorous Instruction %'] \
                 + 0.7 * df_merged['Effective School Leadership %'] \
                 + 0.5 * df_merged['Strong Family-Community Ties %']

Percent ELL
Percent Asian
Percent Black
Percent Hispanic
Percent Black / Hispanic
Percent White
Percent of Students Chronically Absent


In [196]:
drop_columns = ['SED Code', 'Location Code', 'Community School?', 'School Name', 'Grades', 'Grade Low', 
                'School DBN', 'Zip', 'Address (Full)', 'School Category', 'Other Location Code in LCGMS', 
                'Adjusted Grade', 'New?', 'Strong Family-Community Ties Rating', 'Trust Rating', 
                'Student Achievement Rating', 'Effective School Leadership Rating', 'Longitude', 'Latitude', 
                'Supportive Environment Rating', 'Collaborative Teachers Rating', 'Rigorous Instruction Rating', 
                'School Income Estimate'] + SPI_columns
df_merged = df_merged.drop(columns=drop_columns)

In [197]:
# df_merged = df_merged.dropna()
df_merged = df_merged.fillna(0)

In [198]:
label_encode_columns = ['City', 'District', 'Borough']

label_encoders = []
for col in label_encode_columns:
    label_encoder = LabelEncoder()
    label_encoder.fit(df_merged[col])
    df_merged[col] = label_encoder.transform(df_merged[col].values)
    
    label_encoders.append(label_encoder)

In [199]:
# nan_percentage = df_merged.isnull().mean().sort_values(ascending=False)*100
# columns_to_keep = list(nan_percentage[nan_percentage <= 0].index.values)
# columns_to_keep.append('Number of students who took test')

columns_to_keep = list(df_merged.columns.values)
df_merged = df_merged[columns_to_keep]

In [200]:
df_merged = df_merged.astype('float')
df_merged = df_merged.dropna()

In [201]:
feature_list = list(columns_to_keep)
feature_list.remove('Number of students who took test')

In [202]:
X_train = df_merged[feature_list]
y_train = df_merged['Number of students who took test']

y_train = (y_train > y_train.mean()).astype('float')

In [203]:
model = LogisticRegressionCV(cv=5, n_jobs=-1, tol=1e-10, max_iter=1**10, random_state=seed)
model.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=1,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=7,
           refit=True, scoring=None, solver='lbfgs', tol=1e-10, verbose=0)

In [204]:
preds = model.predict(X_train)
print classification_report(y_train, preds)

             precision    recall  f1-score   support

        0.0       0.89      0.98      0.93       438
        1.0       0.93      0.62      0.74       142

avg / total       0.90      0.89      0.89       580



In [205]:
temp = df_merged[preds==1]
# temp['School Income Estimate'].describe()

In [206]:
model.coef_[0]

array([-4.99893626e-04, -6.10113172e-04, -5.33006111e-04, -5.67183290e-05,
       -1.17526193e-03,  1.69470915e-03, -3.33063469e-03, -4.73340781e-03,
       -8.06856570e-03,  9.81148100e-04, -3.68590577e-03,  0.00000000e+00,
       -5.68307225e-05, -4.58503341e-05,  8.61650492e-04,  4.44851934e-04,
        0.00000000e+00,  7.32256051e-05,  1.98695971e-06,  2.32337599e-04,
        1.26419729e-04,  5.38182563e-06, -1.37460703e-06,  1.92262726e-04,
        9.09999244e-04,  9.55230129e-04, -4.28419677e-06,  2.04408779e-04,
       -3.21160343e-05,  5.28466756e-04,  2.04569964e-04,  4.96166751e-06,
        5.62821321e-06,  5.93383373e-04,  1.03070171e-03,  8.17233962e-04,
       -1.49945778e-06,  1.68493288e-04, -7.38864050e-06,  3.59992918e-04,
        2.18319462e-04,  1.12736291e-05,  2.65260163e-06,  3.50388622e-04,
        1.06645327e-03,  1.06653412e-03, -5.99783112e-06,  2.09829704e-04,
        1.08659847e-05,  5.07664281e-04,  2.52815400e-04,  1.25954161e-05,
        2.18051556e-05,  

In [208]:
feature_importance = model.coef_[0]

feature_importance_dict = dict(zip(feature_list, feature_importance))
feature_importance_dict = sorted(feature_importance_dict.iteritems(), key=lambda (k, v): (abs(v), k), reverse=True)
feature_importance_dict

[('SPI', -0.010905640363396732),
 ('Percent Black / Hispanic', -0.008068565699144045),
 ('Grade 6 Math 4s - All Students', 0.004740951590366564),
 ('Percent Hispanic', -0.004733407812306094),
 ('Grade 6 ELA - All Students Tested', 0.003824813097030612),
 ('Grade 6 Math - All Students Tested', 0.003782315374296638),
 ('Student Attendance Rate', -0.0036859057670690404),
 ('Grade 7 Math - All Students Tested', 0.0035997765492203995),
 ('Grade 7 Math 4s - All Students', 0.003598663771591201),
 ('Grade 7 ELA - All Students Tested', 0.0035846835638423573),
 ('Percent Black', -0.003330634694264705),
 ('Grade 6 ELA 4s - All Students', 0.003220322965210863),
 ('Grade 8 ELA 4s - All Students', 0.002958622145008441),
 ('Grade 8 ELA - All Students Tested', 0.0028784828953764357),
 ('Grade 7 ELA 4s - All Students', 0.002836334821294208),
 ('Grade 6 Math 4s - Economically Disadvantaged', 0.0023071222012222537),
 ('Grade 6 Math 4s - Asian or Pacific Islander', 0.0021137922257732796),
 ('Grade 7 Math 

In [209]:
feature_importance = np.abs(model.coef_[0])

feature_importance_min = feature_importance.min()
feature_importance_max = feature_importance.max()
feature_importance = (feature_importance - feature_importance_min) / (feature_importance_max - feature_importance_min)

feature_importance_dict = dict(zip(feature_list, feature_importance))
feature_importance_dict = sorted(feature_importance_dict.iteritems(), key=lambda (k, v): (abs(v), k), reverse=True)
feature_importance_dict

[('SPI', 1.0),
 ('Percent Black / Hispanic', 0.7398525377954939),
 ('Grade 6 Math 4s - All Students', 0.4347247325593928),
 ('Percent Hispanic', 0.43403300077573803),
 ('Grade 6 ELA - All Students Tested', 0.35071879959182095),
 ('Grade 6 Math - All Students Tested', 0.34682194243186804),
 ('Student Attendance Rate', 0.33798159890181884),
 ('Grade 7 Math - All Students Tested', 0.33008392256382757),
 ('Grade 7 Math 4s - All Students', 0.3299818856735471),
 ('Grade 7 ELA - All Students Tested', 0.3286999611571504),
 ('Percent Black', 0.30540477984617187),
 ('Grade 6 ELA 4s - All Students', 0.29528967194071704),
 ('Grade 8 ELA 4s - All Students', 0.2712928398903237),
 ('Grade 8 ELA - All Students Tested', 0.26394441770129007),
 ('Grade 7 ELA 4s - All Students', 0.2600796218087268),
 ('Grade 6 Math 4s - Economically Disadvantaged', 0.21155311603395516),
 ('Grade 6 Math 4s - Asian or Pacific Islander', 0.1938255944023177),
 ('Grade 7 Math 4s - Economically Disadvantaged', 0.158741976909757